In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
test_dir= "/content/drive/MyDrive/Datasets/Plant dieases/Test/Test"
train_dir="/content/drive/MyDrive/Datasets/Plant dieases/Train/Train"
val_dir="/content/drive/MyDrive/Datasets/Plant dieases/Validation/Validation"

In [20]:
import tensorflow as tf

In [21]:
# Creating a Dataset for the Training data
train = tf.keras.utils.image_dataset_from_directory(
    train_dir,  # Directory where the Training images are located
    labels = 'inferred', # Classes will be inferred according to the structure of the directory
    label_mode = 'categorical',
    class_names = ['Healthy', 'Powdery', 'Rust'],
    batch_size = 16,    # Number of processed samples before updating the model's weights
    image_size = (256, 256), # Defining a fixed dimension for all images
    shuffle = True,  # Shuffling data
    seed = 42,  # Random seed for shuffling and transformations
    validation_split = 0, # We don't need to create a validation set from the training set
    crop_to_aspect_ratio = True # Resize images without aspect ratio distortion
)

Found 1322 files belonging to 3 classes.


In [22]:
# Creating a dataset for the Test data
test = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels = 'inferred',
    label_mode = 'categorical',
    class_names = ['Healthy', 'Powdery', 'Rust'],
    batch_size = 16,
    image_size = (256, 256),
    shuffle = True,
    seed = 42,
    validation_split = 0,
    crop_to_aspect_ratio = True
)

Found 150 files belonging to 3 classes.


In [23]:
# Creating a dataset for the Test data
validation = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    labels = 'inferred',
    label_mode = 'categorical',
    class_names = ['Healthy', 'Powdery', 'Rust'],
    batch_size = 16,
    image_size = (256, 256),
    shuffle = True,
    seed = 42,
    validation_split = 0,
    crop_to_aspect_ratio = True
)

Found 60 files belonging to 3 classes.


In [24]:
print('\nTraining Dataset:', train)
print('\nTesting Dataset:', test)
print('\nValidation Dataset:', validation)


Training Dataset: <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None))>

Testing Dataset: <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None))>

Validation Dataset: <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None))>


In [25]:


scaler = tf.keras.layers.Rescaling(1./255) # Defining scaler values between 0 to 1
# Rescaling datasets
train = train.map(lambda x, y: (scaler(x), y))
test = test.map(lambda x, y: (scaler(x), y))
validation = validation.map(lambda x, y: (scaler(x), y))

In [26]:
seed=42
# Creating data augmentation pipeline
augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomRotation(
        factor = (-.25, .3),
        fill_mode = 'reflect',
        interpolation = 'bilinear',
        seed = seed),


        tf.keras.layers.RandomBrightness(
        factor = (-.45, .45),
        value_range = (0.0, 1.0),
        seed = seed),

        tf.keras.layers.RandomContrast(
        factor = (.5),
        seed = seed)
    ]
)

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense

# Initiating model without strategy.scope()
model = Sequential()
model.add(augmentation)  # Assuming you defined an augmentation layer earlier

# Feature Learning Layers
model.add(Conv2D(32,                  # Number of filters/Kernels
                 (3,3),               # Size of kernels (3x3 matrix)
                 strides = 1,         # Step size for sliding the kernel across the input (1 pixel at a time).
                 padding = 'same',    # 'Same' ensures that the output feature map has the same dimensions as the input by padding zeros around the input.
                input_shape = (256,256,3) # Input image shape
                ))

model.add(Activation('relu'))  # Activation function
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(64, (5,5), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.3))

model.add(Conv2D(256, (5,5), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.3))

model.add(Conv2D(512, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.3))

# Flattening tensors
model.add(Flatten())

# Fully-Connected Layers
model.add(Dense(2048))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(3, activation='softmax'))  # Classification layer


In [28]:
# Compiling model

from keras.optimizers import RMSprop

optimizer = RMSprop(learning_rate=0.001)  # Or whatever optimizer you're using
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [32]:


# Defining an Early Stopping and Model Checkpoints
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=5, # Metric to monitor (e.g., validation loss)
    mode='max',                 # Mode to monitor ('min' for loss, 'max' for accuracy)
    restore_best_weights=True    # Restore model weights from the epoch with the best value of the monitored metric
)
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model.keras',     # Path where the model file will be saved
    monitor='val_accuracy',           # Metric to monitor (e.g., validation loss)
    save_best_only=True,           # Save only the best model based on the monitored                      # Verbosity mode (1 for progress messages)
)


In [34]:
history = model.fit(
    train,                           # Training dataset
    validation_data=validation,      # Validation dataset
    epochs=50,                       # Number of epochs
    callbacks=[early_stopping,checkpoint]
)



Epoch 1/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 63s 755ms/step - accuracy: 0.8384 - loss: 0.7194 - val_accuracy: 0.9000 - val_loss: 1.1457
Epoch 2/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 59s 714ms/step - accuracy: 0.8865 - loss: 0.7805 - val_accuracy: 0.9167 - val_loss: 0.3589
Epoch 3/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 61s 737ms/step - accuracy: 0.8742 - loss: 0.6856 - val_accuracy: 0.6333 - val_loss: 0.6214
Epoch 4/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 64s 769ms/step - accuracy: 0.8514 - loss: 2.1768 - val_accuracy: 0.8667 - val_loss: 1.1831
Epoch 5/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 61s 735ms/step - accuracy: 0.8611 - loss: 0.9192 - val_accuracy: 0.7167 - val_loss: 2.1741


In [40]:
# After training, manually save the model in .h5 format
tf.keras.models.save_model(model, 'best_model.keras')


In [42]:
model_1 = tf.keras.models.load_model('best_model.keras')  # or 'best_model.keras'


In [43]:
model_1.predict(test)

10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 592ms/step


array([[3.04710881e-07, 9.99650955e-01, 3.48742440e-04],
       [3.40158812e-09, 1.00000000e+00, 1.71613723e-10],
       [1.00000000e+00, 1.18986167e-20, 9.47157070e-15],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [7.28952330e-07, 9.99999166e-01, 1.04603536e-07],
       [1.00000000e+00, 0.00000000e+00, 2.18345885e-38],
       [3.04485187e-02, 9.69235361e-01, 3.16099758e-04],
       [3.36413994e-03, 9.72845316e-01, 2.37905644e-02],
       [9.99999881e-01, 1.56513996e-37, 7.69878881e-08],
       [1.30405696e-03, 9.98644531e-01, 5.14770909e-05],
       [1.00000000e+00, 3.75300890e-33, 7.99677245e-20],
       [7.99424648e-01, 1.98638231e-01, 1.93717564e-03],
       [4.89866704e-01, 4.95537937e-01, 1.45953679e-02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [1.00146608e-04, 9.99897242e-01, 2.64937626e-06],
       [1.13450681e-22, 1.99954546e-24, 1.00000000e+00],
       [0.00000000e+00, 0.00000

In [49]:

from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('best_model.keras')

# Load and preprocess the image
img_path = '/content/drive/MyDrive/Datasets/Plant dieases/Test/Test/Healthy/8def4d91382175c3.jpg'  # Replace with your image path
img = image.load_img(img_path, target_size=(256, 256))  # Match your model's input size
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Apply the same normalization




In [50]:
# Define class names
class_names = ['Healthy', 'Powdery', 'Rust']

# Predict the class index
predictions = model.predict(img_array)
predicted_class_index = np.argmax(predictions, axis=1)

# Get the corresponding class name
predicted_class_name = class_names[predicted_class_index[0]]  # Use [0] to get the single value from array

# Print the predicted class name
print("Predicted Class:", predicted_class_name)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
Predicted Class: Healthy
